<a href="https://colab.research.google.com/github/sithin42/INT-PROSTATE-Contour-Stability/blob/main/3_StabilityAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

ROOT_PATH = "./"
#Loading the example data from github
if IN_COLAB:
  ROOT_PATH = "./INT-PROSTATE-Contour-Stability"
  !git clone https://github.com/sithin42/INT-PROSTATE-Contour-Stability.git
  import sys
  sys.path.append(ROOT_PATH)
  

Cloning into 'INT-PROSTATE-Contour-Stability'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 66 (delta 24), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (66/66), done.


In [2]:
#Requirements 
!pip install pandas
!pip install pingouin
!pip install seaborn


     |████████████████████████████████| 206 kB 4.0 MB/s 
     |████████████████████████████████| 28.5 MB 49 kB/s 
     |████████████████████████████████| 9.5 MB 50.6 MB/s 
  Created wheel for pingouin: filename=pingouin-0.4.0-py3-none-any.whl size=206904 sha256=2203055a3f73bf3016fa16113602916a2970b36cdbbb433743f9e6a0a1c78b69
  Stored in directory: /root/.cache/pip/wheels/8a/9d/c4/11677c2c4d5c0df9d1d4aa61528d868bb96e13b898e3f51c95
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7048 sha256=006cb1947a59dbe47fa29b10cb62241ca377fb2c1eba571a0aafbe8ec5170c9f
  Stored in directory: /root/.cache/pip/wheels/d6/64/cd/32819b511a488e4993f2fab909a95330289c3f4e0f6ef4676d
Successfully built pingouin littleutils
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmo

In [20]:
import os
from tqdm import tqdm
import pandas as pd
import pingouin as pg
import seaborn as sns
import numpy as np

from ipywidgets import widgets, interact
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [45]:
AUG_TYPE = "in_plane"
BIAS_TYPE = "random"

# Data IO

In [6]:
OUT_PATH = f"./results"

if not os.path.exists(OUT_PATH):
  OUT_PATH = os.path.join(ROOT_PATH,"results")

In [46]:
org_df = pd.read_csv(os.path.join(OUT_PATH,"org_feats.csv"))
aug_df = pd.read_csv(os.path.join(OUT_PATH,f"{AUG_TYPE}_{BIAS_TYPE}","aug_feats.csv"))

merged_df = pd.concat([org_df,aug_df],ignore_index=True)

merged_df.head()

,Unnamed: 0,diagnostics_Versions_PyRadiomics,diagnostics_Versions_Numpy,diagnostics_Versions_SimpleITK,diagnostics_Versions_PyWavelet,diagnostics_Versions_Python,diagnostics_Configuration_Settings,diagnostics_Configuration_EnabledImageTypes,diagnostics_Image-original_Hash,diagnostics_Image-original_Dimensionality,diagnostics_Image-original_Spacing,diagnostics_Image-original_Size,diagnostics_Image-original_Mean,diagnostics_Image-original_Minimum,diagnostics_Image-original_Maximum,diagnostics_Mask-original_Hash,diagnostics_Mask-original_Spacing,diagnostics_Mask-original_Size,diagnostics_Mask-original_BoundingBox,diagnostics_Mask-original_VoxelNum,diagnostics_Mask-original_VolumeNum,diagnostics_Mask-original_CenterOfMassIndex,diagnostics_Mask-original_CenterOfMass,diagnostics_Image-interpolated_Spacing,diagnostics_Image-interpolated_Size,diagnostics_Image-interpolated_Mean,diagnostics_Image-interpolated_Minimum,diagnostics_Image-interpolated_Maximum,diagnostics_Mask-interpolated_Spacing,diagnostics_Mask-interpolated_Size,diagnostics_Mask-interpolated_BoundingBox,diagnostics_Mask-interpolated_VoxelNum,diagnostics_Mask-interpolated_VolumeNum,diagnostics_Mask-interpolated_CenterOfMassIndex,diagnostics_Mask-interpolated_CenterOfMass,diagnostics_Mask-interpolated_Mean,diagnostics_Mask-interpolated_Minimum,diagnostics_Mask-interpolated_Maximum,original_shape_Elongation,original_shape_Flatness,...,exponential_glrlm_ShortRunHighGrayLevelEmphasis,exponential_glrlm_ShortRunLowGrayLevelEmphasis,exponential_glszm_GrayLevelNonUniformity,exponential_glszm_GrayLevelNonUniformityNormalized,exponential_glszm_GrayLevelVariance,exponential_glszm_HighGrayLevelZoneEmphasis,exponential_glszm_LargeAreaEmphasis,exponential_glszm_LargeAreaHighGrayLevelEmphasis,exponential_glszm_LargeAreaLowGrayLevelEmphasis,exponential_glszm_LowGrayLevelZoneEmphasis,exponential_glszm_SizeZoneNonUniformity,exponential_glszm_SizeZoneNonUniformityNormalized,exponential_glszm_SmallAreaEmphasis,exponential_glszm_SmallAreaHighGrayLevelEmphasis,exponential_glszm_SmallAreaLowGrayLevelEmphasis,exponential_glszm_ZoneEntropy,exponential_glszm_ZonePercentage,exponential_glszm_ZoneVariance,exponential_ngtdm_Busyness,exponential_ngtdm_Coarseness,exponential_ngtdm_Complexity,exponential_ngtdm_Contrast,exponential_ngtdm_Strength,exponential_gldm_DependenceEntropy,exponential_gldm_DependenceNonUniformity,exponential_gldm_DependenceNonUniformityNormalized,exponential_gldm_DependenceVariance,exponential_gldm_GrayLevelNonUniformity,exponential_gldm_GrayLevelVariance,exponential_gldm_HighGrayLevelEmphasis,exponential_gldm_LargeDependenceEmphasis,exponential_gldm_LargeDependenceHighGrayLevelEmphasis,exponential_gldm_LargeDependenceLowGrayLevelEmphasis,exponential_gldm_LowGrayLevelEmphasis,exponential_gldm_SmallDependenceEmphasis,exponential_gldm_SmallDependenceHighGrayLevelEmphasis,exponential_gldm_SmallDependenceLowGrayLevelEmphasis,id,dice,judge
0,0,v3.0.1,1.19.5,2.1.1,1.1.1,3.7.12,"{'minimumROIDimensions': 2, 'minimumROISize': ...","{'Original': {}, 'LoG': {'sigma': [2.0, 3.0, 4...",3113612f2899bbba113384f1747b9b716ad30be8,3D,"(0.3125, 0.3125, 3.0)","(512, 512, 30)",391.272048,0.0,11139.0,58c60672e6a2abd8d3caa3ca421d0f37989a4b73,"(0.3125, 0.3125, 3.0)","(512, 512, 30)","(180, 193, 7, 154, 103, 12)",94679,1,"(253.96146980851086, 247.28200551336622, 13.49...","(1.66226046262058, -12.186775620823056, -10.49...","(0.3, 0.3, 3.3)","(185, 131, 28)",58.454832,-154.803072,1207.870512,"(0.3, 0.3, 3.3)","(185, 131, 28)","(12, 12, 6, 156, 107, 11)",89917,1,"(89.24102227609907, 68.21153953090072, 11.9548...","(1.5653578302906581, -12.305190484479791, -11....",136.385329,-74.360766,823.600007,0.819156,0.723521,...,0.438192,0.167660,3.000000,0.272727,2.082645,10.545455,7.232966e+08,7.233929e+08,7.232736e+08,0.236591,1.181818,0.107438,0.212829,3.783304,0.032447,3.459432,0.000122,6.564779e+08,34.178910,0.002928,0.186449,1.017405e-05,0.141323,1.920327,41446.444643,0.460941,26.014993,88484.738359,0.017696,1

#Compute ICC(1,1)

In [47]:
NON_FEAT_COLS = ["diagnostics","id","judge","Unnamed","dice"]
STABILITY_THRESHOLD = 0.90

In [48]:
def compute_icc(df, features):

  out_icc = {"feature":[],"icc_value":[],"ci_down":[],"ci_up":[]}
  pbar = tqdm(range(len(features)),position=0,desc="Computing ICC")

  for feature in features:

      icc = pg.intraclass_corr(data=df,targets='id',raters='judge',ratings=feature)
      
      icc_value = np.round(icc['ICC'][0],2)
      ci_down = np.round(icc['CI95%'][0][0],2)
      ci_up = np.round(icc['CI95%'][0][1],2)

      if ci_down >

      out_icc["feature"].append(feature)
      out_icc["icc_value"].append(icc_value if icc_value>=0 else 0)
      out_icc["ci_down"].append(ci_down if ci_down>=0 else 0)
      out_icc["ci_up"].append(ci_up if ci_up>=0 else 0)
      pbar.update()


  out_icc = pd.DataFrame.from_dict(out_icc)

  return out_icc


In [49]:
def getICC(df):

    features = list(df.columns.values)

    for column in df.columns:
        for ignore_column in NON_FEAT_COLS:
            if ignore_column in column:
                features.remove(column)
    
    out_icc = compute_icc(df,features)

    return out_icc, features
    


In [50]:
out_icc, features = getICC(merged_df)

out_icc.head()

Computing ICC:  10%|█         | 164/1595 [00:11<01:42, 13.90it/s]/usr/local/lib/python3.7/dist-packages/pingouin/reliability.py:364: RuntimeWarning: invalid value encountered in double_scalars
  n * msb)
Computing ICC:  12%|█▏        | 198/1595 [00:14<01:44, 13.39it/s]/usr/local/lib/python3.7/dist-packages/pingouin/reliability.py:364: RuntimeWarning: invalid value encountered in double_scalars
  n * msb)
Computing ICC:  14%|█▎        | 218/1595 [00:15<01:34, 14.63it/s]/usr/local/lib/python3.7/dist-packages/pingouin/reliability.py:364: RuntimeWarning: invalid value encountered in double_scalars
  n * msb)
Computing ICC:  16%|█▋        | 260/1595 [00:18<01:31, 14.59it/s]/usr/local/lib/python3.7/dist-packages/pingouin/reliability.py:364: RuntimeWarning: invalid value encountered in double_scalars
  n * msb)
Computing ICC:  18%|█▊        | 284/1595 [00:20<01:35, 13.76it/s]/usr/local/lib/python3.7/dist-packages/pingouin/reliability.py:364: RuntimeWarning: invalid value encountered in double

,feature,icc_value,ci_down,ci_up
0,original_shape_Elongation,0.57,0.16,1.0
1,original_shape_Flatness,0.98,0.89,1.0
2,original_shape_LeastAxisLength,0.85,0.49,1.0
3,original_shape_MajorAxisLength,0.98,0.89,1.0
4,original_shape_Maximum2DDiameterColumn,0.92,0.66,1.0


#Analysis

#Visualization

In [ ]:
#Interactive Visualization

sitk_img = sitk.ReadImage(img_path)
sitk_mask = sitk.ReadImage(mask_path)

img_arr = sitk.GetArrayFromImage(sitk_img)#Z,X,Y
mask_arr = sitk.GetArrayFromImage(sitk_mask)

spacing_W, spacing_H, _ = sitk_img.GetSpacing()

def visualize(i):
    
    plt.imshow(img_arr[i],cmap='gray')
    if mask_arr[i].sum()>0:
        plt.contour(mask_arr[i])
    plt.show()
    
interact(visualize, i=widgets.IntSlider(len(mask_arr)//2,0,len(mask_arr),1))

#Classical Radiomics Feature Extraction

In [ ]:
PATIENT_IDS = ["PCAMPMRI-00001","PCAMPMRI-00002"]
PARAM_SETTINGS = os.path.join(ROOT_PATH,"paramSettings/StudySettings3D.yaml")

In [ ]:
extractor = radiomics.featureextractor.RadiomicsFeatureExtractor(PARAM_SETTINGS,verbosity=False)

In [ ]:
def extract_features(pids):

  features = []

  pbar = tqdm(range(len(PATIENT_IDS)), desc="Extracting Features")

  for id in pids:

    img_path = os.path.join(ROOT_PATH,"data",id,"image.nii.gz")
    mask_path = os.path.join(ROOT_PATH,"data",id,"mask.nii.gz")

    featureVector = extractor.execute(img_path,mask_path)

    featureVector["id"] = id
    featureVector["dice"] = 1.0 #because this is the ground truth ROI
    featureVector["judge"] = 0

    features.append(featureVector)

    pbar.update()

  df = pd.DataFrame(features)

  return df


In [ ]:
df = extract_features(PATIENT_IDS)
if not os.path.exists("./results"):
  os.makedirs("./results")
df.to_csv("./results/org_feats.csv")
df.head()

#In-Silico Contour Generation & Feature Extraction

In [ ]:

AUG_COUNT = 15
AUG_TYPE = "in_plane"
BIAS_TYPE = "random"

IN_AUG_PARAMS = {'w_spacing':None,"h_spacing":None,'w_stdMM':2.7,'h_stdMM':2.7, 'angle':5,'bias_type':None}
OUT_AUG_PARAMS = {'scale_a':0.6,'scale_b':0.8,'angle':5,'delta_z':2}

PATIENT_IDS = ["PCAMPMRI-00001","PCAMPMRI-00002"]
PARAM_SETTINGS = os.path.join(ROOT_PATH,"paramSettings/StudySettings3D.yaml")

In [ ]:
extractor = radiomics.featureextractor.RadiomicsFeatureExtractor(PARAM_SETTINGS,verbosity=False)

In [ ]:
def extract_aug_features(pids, aug_type, bias_type, aug_count):
    
    features = []

    pbar = tqdm(range(aug_count*len(pids)), desc="Extracting Features", position=0)

    for pid in pids:

      img = sitk.ReadImage(os.path.join(ROOT_PATH,"data",pid,"image.nii.gz"))
      mask = sitk.ReadImage(os.path.join(ROOT_PATH,"data",pid,"mask.nii.gz"))

      img_arr = sitk.GetArrayFromImage(img)
      mask_arr = sitk.GetArrayFromImage(mask)

      aug_fn = get_aug_fn(aug_type, bias_type, img.GetSpacing(), IN_AUG_PARAMS, OUT_AUG_PARAMS)
      subject = tio.Subject(img=tio.ScalarImage(tensor=img_arr[np.newaxis,...]),mask=tio.LabelMap(tensor=mask_arr[np.newaxis,...]))

      for i in range(aug_count):

          pbar.set_description(f"Extracting Features {pid} - Synthetic ROI #{i+1}")

          aug_subject = aug_fn(subject)
          aug_mask_arr = aug_subject["mask"]["data"][0].numpy()

          dice = vol_dice_score(aug_mask_arr,mask_arr)

          aug_mask = sitk.GetImageFromArray(aug_mask_arr)
          aug_mask.SetSpacing(img.GetSpacing())
          aug_mask.SetOrigin(img.GetOrigin())

          featureVector = extractor.execute(img,aug_mask)

          featureVector['id'] = pid
          featureVector['dice'] = dice
          featureVector['judge'] = i+1

          features.append(featureVector)
          pbar.update()
            
                    
    aug_df = pd.DataFrame(features)

    return aug_df


In [ ]:
aug_df = extract_aug_features(PATIENT_IDS,AUG_TYPE,BIAS_TYPE,AUG_COUNT)

if not os.path.exists("./results"):
  os.makedirs("./results")
aug_df.to_csv("./results/aug_feats.csv")
aug_df.head()